## YBS4002

## Marka İtibar ve Şikayet Analizi

####Kütüphanelerin Import Edilmesi

In [ ]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


from warnings import filterwarnings
filterwarnings('ignore')

####Verinin Import Edilmesi

In [ ]:
import pandas as pd 
data = pd.read_csv("HepsiYorunSil.csv")

In [ ]:
data.head()

,Rating,Review,Unnamed: 2
0,1,Ürün bugün elime geçti çok fazla inceleme fırs...,NaN
1,1,Almaya karar verdim. Hemencecik geldi. Keyifle...,NaN
2,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...,NaN
3,1,gayet güzel,NaN
4,1,Çok kaliteli bir ürün ve fiyatı da uygun. Uzun...,NaN


####İsimsiz Kolonların Silinmesi

In [ ]:
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [ ]:
data.head()

,Rating,Review
0,1,Ürün bugün elime geçti çok fazla inceleme fırs...
1,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
2,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
3,1,gayet güzel
4,1,Çok kaliteli bir ürün ve fiyatı da uygun. Uzun...


In [ ]:
sentiment= data["Rating"]
data["Sentiment"]= sentiment
data.head()

,Rating,Review,Sentiment
0,1,Ürün bugün elime geçti çok fazla inceleme fırs...,1
1,1,Almaya karar verdim. Hemencecik geldi. Keyifle...,1
2,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...,1
3,1,gayet güzel,1
4,1,Çok kaliteli bir ürün ve fiyatı da uygun. Uzun...,1


In [ ]:
data["Sentiment"].replace(0, value = "negatif", inplace = True)
data["Sentiment"].replace(1, value = "pozitif", inplace = True)

In [ ]:
data

,Rating,Review,Sentiment
0,1,Ürün bugün elime geçti çok fazla inceleme fırs...,pozitif
1,1,Almaya karar verdim. Hemencecik geldi. Keyifle...,pozitif
2,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...,pozitif
3,1,gayet güzel,pozitif
4,1,Çok kaliteli bir ürün ve fiyatı da uygun. Uzun...,pozitif
...,...,...,...
492,1,Ürün gayet güzel.Işığı oldukça kuvvetli.,pozitif
493,1,"İndirimle 32TL'ye geldi. Küçük ve hafif, avuç ...",pozitif
494,1,Boyutuna göre ışık gücü harika..,pozitif
495,1,bu paraya alabileceginiz en iyi ürün gercekten...,pozitif


In [ ]:
#data.groupby("Sentiment").count()

In [ ]:
df = pd.DataFrame()
df["text"] = data["Review"]
df["label"] = data["Sentiment"]

In [ ]:
df.head()

,text,label
0,Ürün bugün elime geçti çok fazla inceleme fırs...,pozitif
1,Almaya karar verdim. Hemencecik geldi. Keyifle...,pozitif
2,Günlük kullanımınızı çok çok iyi karsılıyor kı...,pozitif
3,gayet güzel,pozitif
4,Çok kaliteli bir ürün ve fiyatı da uygun. Uzun...,pozitif


###NLP Uygulamaları

####Doğal Dil İşleme Kütüphanesinin Import Edilmesi

In [ ]:
pip install nltk

#####Büyük-Küçük Harf Dönüşümü

In [ ]:
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#####Noktalama İşaretlerinin Silinmesi

In [ ]:
df['text'] = df['text'].str.replace('[^\w\s]','')

####Sayıların Silinmesi

In [ ]:
df['text'] = df['text'].str.replace('\d','')

####Stopwords'lerin Silinmesi

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('turkish')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


####Seyrek Geçen Kelimelerin Silinmesi

In [ ]:
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))

####Lemmatizasyon

In [ ]:
#from textblob import Word
#nltk.download('wordnet')
#df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

#!pip install zeyrek
#import zeyrek
#analyzer = zeyrek.MorphAnalyzer()
#df['text'] = df['text'].apply(lambda x: " ".join([analyzer.lemmatize('text') for word in x.split()])) 

In [ ]:
#buyuk-kucuk donusumu
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')
#sayılar
df['text'] = df['text'].str.replace('\d','')
#stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('turkish')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
#seyreklerin silinmesi
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
#lemmi
import zeyrek
analyzer = zeyrek.MorphAnalyzer()
df['text'] = df['text'].apply(lambda x: " ".join([analyzer.lemmatize('text') for word in x.split()])) 
#
from textblob import Word
nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,text,label
0,ürün bugün elime geçti fazla fırsatım olmadı a...,pozitif
1,almaya karar verdim geldi keyifle kullanıyorum,pozitif
2,günlük iyi kısaca mükemmel,pozitif
3,gayet güzel,pozitif
4,kaliteli bir ürün fiyatı uygun uzun sonucu ald...,pozitif


In [ ]:
df.iloc[0]

text     ürün bugün elime geçti fazla fırsatım olmadı a...
label                                              pozitif
Name: 0, dtype: object

####Veriseti'nin Eğitim ve Test olarak Bölünmesi

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df["text"],
                                                                   df["label"], 
                                                                    random_state = 1)

In [ ]:
train_y[0:5]

493    pozitif
233    pozitif
80     pozitif
46     pozitif
142    pozitif
Name: label, dtype: object

####Veri'nin Sayısallaştırılması(Label Encoding)

In [ ]:
encoder = preprocessing.LabelEncoder()

train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [ ]:
train_y[0:5]

array([1, 1, 1, 1, 1])

In [ ]:
test_y[0:5]

array([1, 0, 1, 1, 1])

####Count Vectorizer

Metinde geçen kelime frekanslarının vektöre dönüştürülmesi

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [ ]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [ ]:
x_train_count

<372x2168 sparse matrix of type '<class 'numpy.int64'>'
	with 6025 stored elements in Compressed Sparse Row format>

In [ ]:
vectorizer.get_feature_names()[0:5]

['aaa', 'abartma', 'abime', 'actıgımda', 'acıtmayan']

In [ ]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

####TF-IDF Vectorizer

###### Terim Frekansının ve Belge Frekansının Vektörize Edilmesi 

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

In [ ]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer()

In [ ]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [ ]:
tf_idf_word_vectorizer.get_feature_names()[0:5]

['aaa', 'abartma', 'abime', 'actıgımda', 'acıtmayan']

In [ ]:
x_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

####N-gram level tf-idf

In [ ]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(ngram_range=(2, 3))

####Vektörize Edilen verilerin Eğitim ve Test verisi olarak Bölünmesi

In [ ]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

In [ ]:
#

In [ ]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [ ]:
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

####Lojistik Regresyon

In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8474358974358974


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8397435897435898


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.8397435897435898


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.8397435897435898


####Naive Bayes

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8474358974358974


In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8397435897435898


In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.8397435897435898


In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.8397435897435898


####Random Forest

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8474358974358974


####XGBoost

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.7756410256410255


In [ ]:
#Tahmin

####Tahminlemeler

In [ ]:
lin = linear_model.LogisticRegression()
lin = lin.fit(x_train_count, train_y)

In [ ]:
yeni_yorum = pd.Series("Tek kelimeyle berbat beğenmedim Tek kelimeyle berbat beğenmedim")

yeni_yorum2= pd.Series("	Ürün bugün elime geçti çok fazla inceleme fırsatım olmadı ancak gözüme çarpan ilk şey usb bağlantısı için pil kapağının altında çok güzel yer yapmış olmaları, daha önceden o aparatı kaybetmiş biri olarak bu özelliğini çok sevdim. Ayrıca 1000dpi olması çok iyi bu tip farelerin çoğu 800dpi oluyor. Eğer sizde benim gibi masaüstü bilgisayarınızdan oyuncu faresi ile yüksek dpi ayarına alışmış biriyseniz windows ayarından fareyi en hızlıya getirin ve bu fareyi sipariş verin pişman olmazsınız. Teşekkürler Logitech")

In [ ]:
v = CountVectorizer()
v.fit(train_x)
yeni_yorum = v.transform(yeni_yorum)
yeni_yorum2 = v.transform(yeni_yorum2)


In [ ]:
lin.predict(yeni_yorum)

array([0])

In [ ]:
lin.predict(yeni_yorum2)

array([1])